# Assignment Set 1

*Authors*: Myriam Belkhatir, Salomé Poulain, Shania Sinha

## Import Libraries

In [263]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

%matplotlib inline

## B: Vibrating String (Time Stepping Implementation)

In [2]:
import src.set_1.vibrating_string
reload(src.set_1.vibrating_string)
from src.set_1.vibrating_string import VibratingString

### i. Wave Equation $\Psi(x,t=0) = \sin(2\pi x)$

In [ ]:
def psi_sin_2pi(x):
    return np.sin(2 * np.pi * x)

string1 = VibratingString(psi_sin_2pi, N=100, simulation_time=1, fig_name="sin_2pi")
string1.run_time_stepping()
string1.plot_static_simulation()

### ii. Wave Equation $\Psi(x,t=0) = \sin(5\pi x)$

In [ ]:
def psi_sin_5pi(x):
    return np.sin(5 * np.pi * x)

string2 = VibratingString(psi_sin_5pi, N=100, simulation_time=1, fig_name="sin_5pi")
string2.run_time_stepping()
string2.plot_static_simulation()

### iii. Wave Equation $\Psi(x,t=0) = \sin(5\pi x)$ (Localized) 

The wave equation is localized between x = 1/5 and x = 2/5, else it is 0.

In [ ]:
def psi_localized(x):
    return np.where((x > 1/5) & (x < 2/5), np.sin(5 * np.pi * x), 0)

string3 = VibratingString(psi_localized, N=100, simulation_time=1, fig_name="localized")
string3.run_time_stepping()
string3.plot_static_simulation()

### Create specific 2x2 graph for report

In [ ]:
# Create strings and run simulations
strings = [
    ("sin(2πx)", string1),
    ("sin(5πx)", string2),
    ("sin(5πx) where (1/5 < x < 2/5)", string3)
]

for _, string in strings:
    string.run_time_stepping()

# Plot 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 12), sharey=True)
colormap = plt.cm.plasma
num_steps = 10

for idx, (title, string) in enumerate(strings):
    ax = axes[idx // 2, idx % 2]
    step_size = int(len(string.u) / num_steps)
    norm = plt.Normalize(vmin=0, vmax=num_steps)

    for i, t in enumerate(range(0, len(string.u), step_size)):
        color = colormap(norm(i))
        line_width = 10 * (1 - i / num_steps) + 2
        ax.plot(string.x, string.u[t, :], color=color, alpha=1, linewidth=line_width)

    color = colormap(norm(num_steps))
    ax.plot(string.x, string.u[-1, :], color=color, alpha=1, linewidth=3)
    ax.set_title(title, fontsize=18, fontweight='bold')
    ax.set_xlabel("Position", fontsize=16)
    ax.set_ylabel("Displacement", fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=14)

# Plot legend in the fourth subplot
legend_ax = axes[1, 1]
legend_ax.axis('off')
for i in range(num_steps + 1):
    legend_ax.plot([], [], color=colormap(norm(i)), label=f"t = {i / num_steps:.1f}s", linewidth=(7 * (1 - i / num_steps) + 3))

legend_ax.legend(loc='center', fontsize=18)

plt.tight_layout()
plt.savefig("results/set_1/wave/2x2_waves.png")
plt.show()

## C: Animated Plots of the Vibrating String

In [ ]:
# Wave equation i
string1.plot_dynamic_simulation()

In [ ]:
# Wave equation ii
string2.plot_dynamic_simulation()

In [ ]:
# Wave equation iii
string3.plot_dynamic_simulation()

## E: The Time Dependent Diffusion Equation (Compare with Analytical Solution)

In [1]:
import src.set_1.time_dependent_diffusion
reload(src.set_1.time_dependent_diffusion)
from src.set_1.time_dependent_diffusion import TimeDependentDiffusion

In [2]:
simulation = TimeDependentDiffusion(N=50, simulation_time=1, fig_name="diffusion_simulation") 
simulation.run_time_stepping()

In [ ]:
simulation.check_analytical_solution()

## F: Two-dimensional Domain Results at Different Times

In [ ]:
simulation.create_subplot_log_time()

## G: Animated Plot of the Time Dependent Diffusion Equation

In [ ]:
simulation.create_animation()

## H: Jacobi Iteration, Gauss-Seidel and Successive Over-Relaxation

The following code is a simple implementation of the Jacobi Iteration, Gauss-Seidel and Successive Over-Relaxation (SOR) methods for solving the 2D Laplace equation. It also compares the analytical solution (as given in Equation 5 in the assignment document) with the numerical solution obtained using the three methods.

In [264]:
# Import modules for each numerical method
import src.set_1.jacobi_iteration_optimized
reload(src.set_1.jacobi_iteration_optimized)
from src.set_1.jacobi_iteration_optimized import JacobiIteration

import src.set_1.gauss_seidel_iteration_optimized
reload(src.set_1.gauss_seidel_iteration_optimized)
from src.set_1.gauss_seidel_iteration_optimized import GaussSeidelIteration

import src.set_1.successive_over_relaxation_optimized
reload(src.set_1.successive_over_relaxation_optimized)
from src.set_1.successive_over_relaxation_optimized import SORIteration

In [ ]:
solver_ji = JacobiIteration(N=50)
iterations_taken_ji, concentration_ji, diffs_ji = solver_ji.solve()
# solver_ji.plot_solution() # Uncomment to plot the solution

In [ ]:
iterations_taken_ji[-1]

In [ ]:
solver_gs = GaussSeidelIteration(N=50)
iterations_taken_gs, concentration_gs, diffs_gs = solver_gs.solve()
# solver_gs.plot_solution() # Uncomment to plot the solution

In [ ]:
iterations_taken_gs[-1]

In [310]:
solver_sor = SORIteration(N=50)
iterations_taken_sor, concentration_sor, diffs_sor = solver_sor.solve(omega=1.9196969696969697)
# solver_sor.plot_solution() # Uncomment to plot the solution

In [ ]:
iterations_taken_sor[-1]

#### Comparing the numerical solutions with the analytical solution which is given by $c(y, t) = y$

In [9]:
def analytical_solution(N):
    """
    Compute the analytical solution for the diffusion equation.
    """
    y = np.linspace(0, 1, N)
    return y

def compare_analytical_solution(N, c_sor, c_gs, c_ji, separate=False):
    """
    Compare the analytical solution with the numerical solution
    """
    # Get the numerical solution at the center of the domain
    analytical = analytical_solution(N)
    numerical_sor = c_sor[:, N // 2]
    numerical_gs = c_gs[:, N // 2]
    numerical_ji = c_ji[:, N // 2]
    
    if separate:
        plt.figsize=(8, 8)
        plt.plot(analytical, linestyle="--", label="Analytical", linewidth=2.5)
        plt.plot(numerical_sor, linestyle="-", label="SOR", marker="o", alpha=0.6, markersize=5)
        plt.plot(numerical_gs, linestyle="-", label="Gauss-Seidel", marker="^", alpha=0.6, markersize=4)
        plt.plot(numerical_ji, linestyle="-", label="Jacobi", marker="s", alpha=0.6, markersize=3)
        plt.title("Analytical vs. Numerical Solutions", fontsize=18, fontweight="bold")
        plt.xlabel("x", fontsize=16)
        plt.ylabel("Concentration (c)", fontsize=16)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.legend()
        plt.grid()
        plt.savefig("results/set_1/numerical_methods/all_analytical_vs_numerical.png")
        plt.show()
    else:
        # Create a 2x2 subplot
        fig, axes = plt.subplots(2, 2, figsize=(12, 12), sharey=True)
        fig.delaxes(axes[1][1])
        axes = axes.flatten()
        for idx, (ax, c, title) in enumerate(zip(axes, [numerical_ji, numerical_gs, numerical_sor], ["Jacobi", "Gauss-Seidel", "SOR"])):
            ax.plot(analytical, linestyle="--", label="Analytical", linewidth=2.5)
            if title == "SOR":
                ax.plot(c, linestyle="-", label=title, marker="o", alpha=0.5, markersize=5)
            elif title == "Gauss-Seidel":
                ax.plot(c, linestyle="-", label=title, marker="^", alpha=0.5, markersize=5, color="green")
            else:
                ax.plot(c, linestyle="-", label=title, marker="s", alpha=0.5, markersize=5, color="red")
            ax.set_title(title, fontsize=16, fontweight="bold")
            ax.set_xlabel("x", fontsize=14)
            ax.set_ylabel("Concentration (c)", fontsize=14)
            ax.tick_params(axis='both', which='major', labelsize=12)
            ax.legend()
            ax.grid()
        plt.tight_layout()
        plt.savefig("results/set_1/numerical_methods/2x2_analytical_vs_numerical.png")
        plt.show()

In [ ]:
compare_analytical_solution(50, concentration_sor, concentration_gs, concentration_ji, separate=True)

In [ ]:
# Plotted separately for better visualization
compare_analytical_solution(50, concentration_sor, concentration_gs, concentration_ji, separate=False)

## I: Convergence Measure vs Number of Iterations

The following code plots the convergence measure against the number of iterations for each of the three solving methods. The convergence measure is defined as:

$$ \delta \equiv \max_{i,j} \left| c_{i,j}^{k+1} - c_{i,j}^{k} \right| < \epsilon $$

where $\epsilon$ is a small number (in this case, $10^{-5}$).

In [29]:
# Plot the number of iterations taken for each method on the x-axis and the diffs on the y-axis
def plot_iterations_vs_diffs(iterations, diffs, labels, colors):
    """
    Plot the number of iterations taken for each method on the x-axis and the diffs on the y-axis
    """
    plt.figure(figsize=(12, 8))
    for iteration, diff, label in zip(iterations, diffs, labels):
        # Create a log-lin plot
        if label == "SOR (omega=1)":
            plt.plot(iteration, diff, label=label, linestyle="--", color=colors.pop())
        else:
            plt.plot(iteration, diff, label=label, color=colors.pop())
    plt.yscale('log')
    plt.title(r"Iterations vs. $\delta$", fontsize=18, fontweight="bold")
    plt.xlabel("Iterations", fontsize=16)
    plt.ylabel(r"$\delta$", fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=16)
    plt.grid()
    plt.savefig("results/set_1/numerical_methods/iterations_vs_diffs.png")
    plt.show()

In [ ]:
# Run SOR for other omega values
omegas = [0.8, 1, 1.6, 1.92]
iterations_sor_all_omegas = []
diffs_sor_all_omegas = []
labels_sor = [f"SOR (omega={omega})" for omega in omegas]

for omega in omegas:
    solver_sor = SORIteration(N=50)
    iterations_taken_sor, concentration_sor, diffs_sor = solver_sor.solve(omega=omega)
    iterations_sor_all_omegas.append(iterations_taken_sor)
    diffs_sor_all_omegas.append(diffs_sor)

In [ ]:
iterations_all_methods = [iterations_taken_ji, iterations_taken_gs]     # Start with the Jacobi and Gauss-Seidel iterations
iterations_all_methods.extend(iterations_sor_all_omegas)                # Extend the list of iterations with the SOR iterations

diffs_all_methods = [diffs_ji, diffs_gs]            # Start with the Jacobi and Gauss-Seidel diffs
diffs_all_methods.extend(diffs_sor_all_omegas)      # Extend the list of diffs with the SOR diffs

labels = ["Jacobi", "Gauss-Seidel"]                 # Start with the Jacobi and Gauss-Seidel labels
labels.extend(labels_sor)                           # Extend the list of labels with the SOR labels

# List of colors for the plot
colors = ["#800000", "#ed7014", "#023020", "#097969", "#088F8F", "#5F9EA0"]

# Plot
plot_iterations_vs_diffs(iterations_all_methods, diffs_all_methods, labels, colors[::-1])

## J: Finding the Optimal $\omega$ for SOR

The following code finds the optimal value of $\omega$ for the SOR method and finds the relationship between grid size (N) and $\omega$.

In [ ]:
# Perform a grid search to find minimum omega for SOR given N = 50
omega_values = np.linspace(1.7, 1.99, 100)
iterations_taken_sor = []
N_values = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
min_omegas = []
for N in N_values:
    for omega in omega_values:
        solver_sor = SORIteration(N=N)
        iter_sor, c_sor, diffs = solver_sor.solve(omega=omega)
        iterations_taken_sor.append(iter_sor[-1])
    min_omega = omega_values[np.argmin(iterations_taken_sor)]
    print(f"Minimum Omega for N = {N}: {min_omega} with {np.min(iterations_taken_sor) + 1} iterations")
    min_omegas.append(min_omega)
    iterations_taken_sor = [] 

In [ ]:
print(min_omegas)
print(N_values)

In [300]:
# Write a function to plot the minimum omega values for different N values
def plot_min_omegas(N_values, min_omegas):
    """
    Plot the minimum omega values for different N values
    """
    plt.figure(figsize=(12, 8))
    plt.plot(N_values, min_omegas, marker="o")
    plt.title("Optimal Omega Values for Different N Values", fontsize=18, fontweight="bold", color="#023020")
    plt.xlabel("N", fontsize=16)
    plt.ylabel("Omega", fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid()
    plt.savefig("results/set_1/numerical_methods/min_omegas_vs_N.png", bbox_inches="tight")
    plt.show()

In [ ]:
# Plot
plot_min_omegas(N_values, min_omegas)

## K: Implementing Sinks

The following code implements sinks in the domain (areas where concentration function is 0). It also investigates the effect of presence of one or more of these sinks on the convergence of the three numerical methods, including it's effect on $\omega$ in the SOR method.

### Jacobi Iteration

In [268]:
import src.set_1.jacobi_with_sinks
reload(src.set_1.jacobi_with_sinks)
from src.set_1.jacobi_with_sinks import JacobiIterationSinks

In [115]:
# Possible activated sinks

# rectangle_small:           Small rectangle
# rectangle_medium:          Medium rectangle
# triangle_right:            Right-angled triangle
# equilateral_upside_down:   Equilateral triangle (grows downward)
# equilateral:               Equilateral triangle (grows upward)
# circle_center:             Circular sink
# circle_through_boundary:   Circular sink (going through the periodic x boundary)

In [ ]:
activated_sinks=["circle_center"]

solver_ji_sinks = JacobiIterationSinks(N=50, activated_objects=activated_sinks)
iterations_taken_ji_sinks, concentration_ji_sinks, diffs_ji_sinks = solver_ji_sinks.solve()
solver_ji_sinks.plot_solution()

In [ ]:
# Plot the number of iterations taken for each method on the y-axis and number of activated sinks on the x-axis
activated_sinks_list = [["circle_center"], ["circle_center", "rectangle_small"], ["circle_center", "rectangle_small", "equilateral"], ["circle_center", "circle_through_boundary", "rectangle_small", "equilateral"]]

iterations_taken_all_sinks_jacobi = []

for activated_sinks in activated_sinks_list:
    solver_ji_sinks = JacobiIterationSinks(N=50, activated_objects=activated_sinks)
    iterations_taken_ji_sinks, concentration_ji_sinks, diffs_ji_sinks = solver_ji_sinks.solve()
    iterations_taken_all_sinks_jacobi.append(len(iterations_taken_ji_sinks))

In [ ]:
# Plot
def plot_iterations_vs_sinks_jacobi(iterations_taken_all_sinks, activated_sinks_list, iter_ji_no_sink):
    """
    Plot the number of iterations taken for each method on the y-axis and number of activated sinks on the x-axis
    """
    xlabels = [f"{len(sinks)} objects" if len(sinks) > 1 else f"{len(sinks)} object" for sinks in activated_sinks_list]

    plt.figure(figsize=(12, 8))

    plt.plot(iterations_taken_all_sinks, marker="o", linestyle="-", label="Jacobi with sinks")
    plt.axhline(y=iter_ji_no_sink, color="maroon", linestyle="--", label=r"Jacobi without sinks (iter=7528)")
    plt.title("Iterations Taken for Jacobi", fontsize=28, fontweight="bold")
    plt.xlabel("Number of Objects", fontsize=16)
    plt.ylabel("Iterations", fontsize=16)
    plt.xticks(range(len(xlabels)), xlabels, fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=16)
    plt.grid()
    plt.savefig("results/set_1/numerical_methods/ji_iterations_sinks_vs_nosinks.png", bbox_inches="tight")
    plt.show()

plot_iterations_vs_sinks_jacobi(iterations_taken_all_sinks_jacobi, activated_sinks_list, len(iterations_taken_ji))

### Gauss-Seidel Iteration

In [250]:
import src.set_1.gauss_seidel_with_sinks
reload(src.set_1.gauss_seidel_with_sinks)
from src.set_1.gauss_seidel_with_sinks import GaussSeidelIterationSinks

In [ ]:
activated_sinks=["circle_center", "rectangle_small"]

solver_gs_sinks = GaussSeidelIterationSinks(N=50, activated_objects=activated_sinks)
iterations_taken_gs_sinks, concentration_gs_sinks, diffs_gs_sinks = solver_gs_sinks.solve()
solver_gs_sinks.plot_solution()

In [ ]:
# Plot the number of iterations taken for each method on the y-axis and number of activated sinks on the x-axis
activated_sinks_list = [["circle_center"], ["circle_center", "rectangle_small"], ["circle_center", "rectangle_small", "equilateral"], ["circle_center", "circle_through_boundary", "rectangle_small", "equilateral"]]

iterations_taken_all_sinks_gs = []

for activated_sinks in activated_sinks_list:
    solver_gs_sinks = GaussSeidelIterationSinks(N=50, activated_objects=activated_sinks)
    iterations_taken_gs_sinks, concentration_gs_sinks, diffs_gs_sinks = solver_gs_sinks.solve()
    iterations_taken_all_sinks_gs.append(len(iterations_taken_gs_sinks))

In [ ]:
# Plot
def plot_iterations_vs_sinks_gs(iterations_taken_all_sinks, activated_sinks_list, iter_gs_no_sink):
    """
    Plot the number of iterations taken for each method on the y-axis and number of activated sinks on the x-axis
    """
    xlabels = [f"{len(sinks)} objects" if len(sinks) > 1 else f"{len(sinks)} object" for sinks in activated_sinks_list]

    plt.figure(figsize=(12, 8))

    plt.plot(iterations_taken_all_sinks, marker="o", linestyle="-", label="Gauss-Seidel with sinks")
    plt.axhline(y=iter_gs_no_sink, color="maroon", linestyle="--", label=r"Gauss-Seidel without sinks (iter=4114)")
    plt.title("Iterations Taken for Gauss-Seidel", fontsize=28, fontweight="bold")
    plt.xlabel("Number of Objects", fontsize=16)
    plt.ylabel("Iterations", fontsize=16)
    plt.xticks(range(len(xlabels)), xlabels, fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=16)
    plt.grid()
    plt.savefig("results/set_1/numerical_methods/gs_iterations_sinks_vs_nosinks.png", bbox_inches="tight")
    plt.show()

plot_iterations_vs_sinks_gs(iterations_taken_all_sinks_gs, activated_sinks_list, len(iterations_taken_gs))

### Successive Over-Relaxation (SOR) Iteration

In [273]:
import src.set_1.successive_over_relaxation_with_sinks
reload(src.set_1.successive_over_relaxation_with_sinks)
from src.set_1.successive_over_relaxation_with_sinks import SORIterationSinks

In [ ]:
activated_sinks=["circle_center"]

solver_sor_sinks = SORIterationSinks(N=50, activated_objects=activated_sinks)
iterations_taken_sor_sinks, concentration_sor_sinks, diffs_sor_sinks = solver_sor_sinks.solve(omega=1.92)
solver_sor_sinks.plot_solution()

In [ ]:
# Run a search to find minimum omega for SOR with sinks given N = 50
omega_values = np.linspace(1.6, 1.99, 100)
iterations_taken_sor_sinks = []
min_iter_each_omega = []
min_omegas_sinks = []
# activated_sinks_list = [["circle_center"], ["rectangle_small"], ["equilateral]"], ["circle_center", "rectangle_small"], ["circle_center", "rectangle_small", "equilateral"], ["circle_center", "circle_through_boundary", "rectangle_small", "equilateral"]]

for activated_sinks in activated_sinks_list:
    for omega in omega_values:
        solver_sor_sinks = SORIterationSinks(N=50, activated_objects=activated_sinks)
        iter_sor_sinks, c_sor_sinks, diffs = solver_sor_sinks.solve(omega=omega)
        iterations_taken_sor_sinks.append(iter_sor_sinks[-1])
    min_omega = omega_values[np.argmin(iterations_taken_sor_sinks)]
    print(f"Minimum Omega for N = 50 with sinks: {min_omega} with {np.min(iterations_taken_sor_sinks) + 1} iterations")
    min_omegas_sinks.append(min_omega)
    min_iter_each_omega.append(np.min(iterations_taken_sor_sinks) + 1)
    iterations_taken_sor_sinks = []

In [ ]:
min_omegas_sinks

In [ ]:
min_iter_each_omega

In [319]:
# Function to plot the number of iterations taken for each method on the y-axis in the presence of multiple sinks
def plot_iterations_sinks_sor(min_iter_each_omega, activated_sinks_list, iter_sor_no_sink):
    """
    Plot the number of iterations taken for each method on the y-axis in the presence of multiple sinks
    """
    xlabels = [f"{len(sinks)} objects" if len(sinks) > 1 else f"{len(sinks)} object" for sinks in activated_sinks_list]

    plt.figure(figsize=(12, 8))

    plt.plot(min_iter_each_omega, marker="o", linestyle="-", label="SOR with sinks")
    plt.axhline(y=iter_sor_no_sink, color="maroon", linestyle="--", label=r"SOR without sinks ($\omega$=1.92, iter=212)")
    plt.title("Iterations Taken for SOR", fontsize=28, fontweight="bold")
    plt.xlabel("Number of Objects", fontsize=16)
    plt.ylabel("Iterations", fontsize=16)
    plt.xticks(range(len(xlabels)), xlabels, fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=16)
    plt.grid()
    plt.savefig("results/set_1/numerical_methods/sor_iterations_sinks_vs_nosinks.png", bbox_inches="tight")
    plt.show()

In [ ]:
plot_iterations_sinks_sor(min_iter_each_omega, activated_sinks_list, len(iterations_taken_sor))

In [321]:
# Function to plot minimum omega values for different sink configurations
def plot_min_omegas_sinks(activated_sinks_list, min_omegas_sinks):
    """
    Plot minimum omega values for different sink configurations
    """
    xlabels = [f"{len(sinks)} objects" if len(sinks) > 1 else f"{len(sinks)} object" for sinks in activated_sinks_list]

    plt.figure(figsize=(12, 8))
    plt.plot(xlabels, min_omegas_sinks, marker="o")
    plt.title("Optimal Omega Values", fontsize=28, fontweight="bold")
    plt.xlabel("Number of Objects", fontsize=16)
    plt.ylabel("Omega", fontsize=16)
    plt.xticks(range(len(xlabels)), xlabels, fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid()
    plt.savefig("results/set_1/numerical_methods/sor_min_omegas_sinks.png", bbox_inches="tight")
    plt.show()

In [ ]:
plot_min_omegas_sinks(activated_sinks_list, min_omegas_sinks)

### All plots together (for the report)

In [294]:
def plot_iterations_vs_sinks(iterations_jacobi, iterations_gauss, iterations_sor, activated_sinks_list, iter_ji_no_sink, iter_gs_no_sink, iter_sor_no_sink):
    """
    Plot the number of iterations taken for Jacobi, Gauss-Seidel, and SOR methods on the y-axis and number of activated sinks on the x-axis.
    """
    xlabels = [f"{len(sinks)} objects" if len(sinks) > 1 else f"{len(sinks)} object" for sinks in activated_sinks_list]

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle("Number of Iterations Taken for Different Numerical Methods with Sink Configurations", fontsize=20, fontweight="bold")

    # Jacobi plot
    axes[0, 0].plot(iterations_jacobi, marker="o", linestyle="-", label="Jacobi with sinks")
    axes[0, 0].axhline(y=iter_ji_no_sink, color="maroon", linestyle="--", label="Jacobi without sinks (iter=7528)")
    axes[0, 0].set_title("Jacobi Method", fontsize=16, fontweight="bold")
    axes[0, 0].set_xlabel("Number of Objects", fontsize=14)
    axes[0, 0].set_ylabel("Iterations", fontsize=14)
    axes[0, 0].set_xticks(range(len(xlabels)))
    axes[0, 0].set_xticklabels(xlabels, fontsize=12)
    axes[0, 0].grid()
    axes[0, 0].legend(fontsize=14)

    # Gauss-Seidel plot
    axes[0, 1].plot(iterations_gauss, marker="o", linestyle="-", label="Gauss-Seidel with sinks")
    axes[0, 1].axhline(y=iter_gs_no_sink, color="maroon", linestyle="--", label="Gauss-Seidel without sinks (iter=4114)")
    axes[0, 1].set_title("Gauss-Seidel Method", fontsize=16, fontweight="bold")
    axes[0, 1].set_xlabel("Number of Objects", fontsize=14)
    axes[0, 1].set_ylabel("Iterations", fontsize=14)
    axes[0, 1].set_xticks(range(len(xlabels)))
    axes[0, 1].set_xticklabels(xlabels, fontsize=12)
    axes[0, 1].grid()
    axes[0, 1].legend(fontsize=14)

    # SOR plot
    axes[1, 0].plot(iterations_sor, marker="o", linestyle="-", label="SOR with sinks")
    axes[1, 0].axhline(y=iter_sor_no_sink, color="maroon", linestyle="--", label=r"SOR without sinks ($\omega$=1.92, iter=212)")
    axes[1, 0].set_title("Successive Over-Relaxation (SOR) Method", fontsize=16, fontweight="bold")
    axes[1, 0].set_xlabel("Number of Objects", fontsize=14)
    axes[1, 0].set_ylabel("Iterations", fontsize=14)
    axes[1, 0].set_xticks(range(len(xlabels)))
    axes[1, 0].set_xticklabels(xlabels, fontsize=12)
    axes[1, 0].grid()
    axes[1, 0].legend(fontsize=14)

    # Legend in the 4th subplot
    # axes[1, 1].legend(loc="center", fontsize=14)
    axes[1, 1].axis("off")

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig("results/set_1/numerical_methods/all_iterations_sinks_vs_nosinks.png")
    plt.show()

In [ ]:
# Example function call with necessary data
plot_iterations_vs_sinks(iterations_taken_all_sinks_jacobi, iterations_taken_all_sinks_gs, min_iter_each_omega, activated_sinks_list, len(iterations_taken_ji), len(iterations_taken_gs), len(iterations_taken_sor))

## Optional: Objects made of Insulating Material in the Domain 

Values inside the insulation remain unchanged. Also, the boundary of the insulation does not contribute to the update of neighboring points.

In [184]:
import src.set_1.jacobi_with_insulation
reload(src.set_1.jacobi_with_insulation)
from src.set_1.jacobi_with_insulation import JacobiIteration

In [ ]:
# Possible activated insulated objects: "rectangle", "circle"
activated_insulated_objects = ["rectangle"]

solver_ji_insulated = JacobiIteration(N=50, activated_objects=activated_insulated_objects)
iterations_taken_ji_insulated, concentration_ji_insulated, diffs_ji_insulated = solver_ji_insulated.solve()
solver_ji_insulated.plot_solution()